# Read the data

In [1]:
import os
import pandas as pd

from fastapi import FastAPI
from typing import List
from models import CountryData, Pillar, Area, Indicator, Metric
import re

/Users/jessiefung/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
filepath = "../data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx" # Specify the correct path to the file
df_assessments = pd.read_excel(filepath)

# Convert the date columns to datetime type so we can filter by year later
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

/var/folders/kv/l9vcg1gs7xb1tfhh37wfc4_w0000gn/T/ipykernel_51287/3951065428.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


# Define the function

In [22]:
country = 'Barbados'
assessment_year = 2024

data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]

#remember which columns are area, indicator, metric
area_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("area")]
indicator_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("indicator")]
metric_cols =  [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("metric")] 

#remove unecessary columns
data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

#rename columns so they align with output
remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
data = data.rename(columns=remap_column_names)


#get flat Pandas series of country data
data = data.iloc[0]
data = data.fillna("")

data

EP.1                             Yes
EP.1.a                           Yes
EP.1.b                       No Data
EP.1.c                           Yes
EP.2                         Partial
                      ...           
CF.4                                
CF.4.i      10.95 MW/US$ billion GDP
CF.4.ii      4.69 MW/US$ billion GDP
CF.4.iii                     No data
CF.4.iv                      No data
Name: 32, Length: 74, dtype: object

In [23]:
#get metric
metrics = [{'name': metric, 'value': data[f'{metric}']} for metric in metric_cols]

#get indicator
indicators = [{'name': indicator, 'assessment': data[f"{indicator}"],
                   'metrics': next((met for met in metrics if met["name"].startswith(indicator)), "")} for indicator in indicator_cols]   

    #get area
areas = [{'name': area, 'assessment': data[f"{area}"],
              'indicators': [ind for ind in indicators if ind["name"].startswith(area)]} for area in area_cols]


#get pillar
pillars = [{'name': pillar, 'areas': areas} for pillar in ["EP","CP","CF"]]

output_dict = {'pillars': [pillar for pillar in pillars]}

output_dict

{'pillars': [{'name': 'EP',
   'areas': [{'name': 'EP.1',
     'assessment': 'Yes',
     'indicators': [{'name': 'EP.1.a', 'assessment': 'Yes', 'metrics': ''},
      {'name': 'EP.1.b', 'assessment': 'No Data', 'metrics': ''},
      {'name': 'EP.1.c', 'assessment': 'Yes', 'metrics': ''}]},
    {'name': 'EP.2',
     'assessment': 'Partial',
     'indicators': [{'name': 'EP.2.a',
       'assessment': 'Yes',
       'metrics': {'name': 'EP.2.a.i', 'value': '-73%'}},
      {'name': 'EP.2.b',
       'assessment': 'No',
       'metrics': {'name': 'EP.2.b.i',
        'value': 'No or unsuitable disclosure'}},
      {'name': 'EP.2.c',
       'assessment': 'No Data',
       'metrics': {'name': 'EP.2.c.i', 'value': 'No Data'}},
      {'name': 'EP.2.d',
       'assessment': 'Yes',
       'metrics': {'name': 'EP.2.d.i', 'value': '-94%'}}]},
    {'name': 'EP.3',
     'assessment': 'Yes',
     'indicators': [{'name': 'EP.3.a',
       'assessment': 'Yes',
       'metrics': {'name': 'EP.3.a.i', 'value': 

In [24]:
CountryData(**output_dict)

ValidationError: 22 validation errors for CountryData
pillars -> 0 -> areas -> 0 -> indicators -> 1 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=No Data; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars -> 0 -> areas -> 0 -> indicators
  unhashable type: 'list' (type=type_error)
pillars -> 0 -> areas -> 1 -> indicators -> 2 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=No Data; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars -> 0 -> areas -> 1 -> indicators
  unhashable type: 'list' (type=type_error)
pillars -> 0 -> areas -> 8 -> indicators -> 3 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=Not Applicable; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars

In [10]:
def get_country_data(country: str, assessment_year: int):
    data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]

    #remember which columns are area, indicator, metric
    area_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("area")]
    indicator_cols = [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("indicator")]
    metric_cols =  [re.sub(".*?\s", "", col) for col in data.columns if col.startswith("metric")] 

    #remove unecessary columns
    data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

    #rename columns so they align with output
    remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
    data = data.rename(columns=remap_column_names)

    #get flat Pandas series of country data
    data = data.iloc[0]
    data = data.fillna("")

    #get metric
    metrics = [{'name': metric, 'value': data[f'{metric}']} for metric in metric_cols]

    #get indicator
    indicators = [{'name': indicator, 'assessment': data[f"{indicator}"],
                   'metrics': next((met for met in metrics if met["name"].startswith(indicator)), "")} for indicator in indicator_cols]   

    #get area
    areas = [{'name': area, 'assessment': data[f"{area}"],
              'indicators': [ind for ind in indicators if ind["name"].startswith(area)]} or "" for area in area_cols]

    #get pillar
    pillars = [{'name': pillar, 'areas': areas} for pillar in ["EP","CP","CF"]]

    output_dict = {'pillars': [pillar for pillar in pillars]}

    output = CountryData(**output_dict)

    return output

In [37]:
unique_values = [df_assessments[col].unique().tolist() for col in df_assessments.columns if col.startswith(("metric"))]

In [38]:
list((set(sum(unique_values, []))))

['16.48 MW/US$ billion GDP',
 '1.21 MW/US$ billion GDP',
 '2.94 MJ/PPP-adjusted GDP',
 '4.80 MW/US$ billion GDP',
 '49%',
 '2.12 MJ/PPP-adjusted GDP',
 '1.36%',
 '3%',
 '63%',
 '2.27 MJ/PPP-adjusted GDP',
 '865%',
 '264%',
 '3.26 MJ/US$ PPP-adjusted GDP',
 '139.67 MW/US$ billion GDP',
 '3.04 MJ/PPP-adjusted GDP',
 '0%',
 '282%',
 '31.56 MW/US$ billion GDP',
 '120.89 MW/US$ billion GDP',
 '0.42%',
 '248%',
 '2035',
 '2.53 MJ/PPP-adjusted GDP',
 '8%',
 '0.79 MW/US$ billion GDP',
 '317%',
 '33.76 MW/US$ billion GDP',
 '84%',
 '22.85 MW/US$ billion GDP',
 '83.43 MW/US$ billion GDP',
 '28%',
 '2.04 MJ/PPP-adjusted GDP',
 '61%',
 '2.33 MW/US$ billion GDP',
 "'-16%",
 '514%',
 '22%',
 '20%',
 '338.29 MW/US$ billion GDP',
 '7.91 MW/US$ billion GDP',
 '-48%',
 '40.11 MW/US$ billion GDP',
 '2.48 MJ/US$ PPP-adjusted GDP',
 '27%',
 '50.44 MW/US$ billion GDP',
 '-26%',
 '0.17 MW/US$ billion GDP',
 '-39%',
 '2060',
 '6.32 MW/US$ billion GDP',
 '2.23 MW/US$ billion GDP',
 '4.41 MW/US$ billion GDP',
 

# Test how the function behaves

In [12]:
get_country_data("Barbados", 2024)

ValidationError: 22 validation errors for CountryData
pillars -> 0 -> areas -> 0 -> indicators -> 1 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=No Data; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars -> 0 -> areas -> 0 -> indicators
  unhashable type: 'list' (type=type_error)
pillars -> 0 -> areas -> 1 -> indicators -> 2 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=No Data; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars -> 0 -> areas -> 1 -> indicators
  unhashable type: 'list' (type=type_error)
pillars -> 0 -> areas -> 8 -> indicators -> 3 -> assessment
  unexpected value; permitted: 'Exempt', 'No', 'Not applicable', 'Partial', 'Yes', '' (type=value_error.const; given=Not Applicable; permitted=('Exempt', 'No', 'Not applicable', 'Partial', 'Yes', ''))
pillars

# Testing out Pydantic Models

In [9]:
from pydantic import BaseModel, Field
from typing import Literal